# Ceneo Scraper

1. Pobranie skłoadowych pojedynczej opinii

In [1]:
selectors = {
    'author': (".user-post__author-name",),
    'recommandation': (".user-post__author-recomendation > em",),
    'rating': (".user-post__score-count",),
    'content': (".user-post__text",),
    'pros': ("div.review-feature__title--positives ~ review-feature__item", None, True),
    'cons': ("div.review-feature__title--negatives ~ review-feature__item", None, True),
    'opinion_date': (".user-post__published > time:nth-child(1)", 'datetime'),
    'purchase_date': (".user-post__published > time:nth-child(2)", 'datetime'),
    'likes': ("button.vote-yes > span",),
    'dislikes': ("button.vote-no > span",),
    'opinion_id': (None, "data-entry-id"),
}

## Struktura opinii

|Składowa|zmienna|selektor|
|--------|-------|--------|
|identyfikator opinii|opinion_id|["data-entry-id"]|
|autor|author|span.user-post__author-name|
|rekomendacje|recommendation|.user-post__author-recomendation > em|
|liczba gwiazdek|starts|span.user-post__score-count|
|tresc opinii|content|div.user-post__text|
|lista zalet|pros|div.review-feature__title--positives ~ div.review-feature__item|
|lista wad|cons|div.review-feature__title--negatives ~ div.review-feature__item|
|data wystawienia opinii|opinion_date|span.user-post__published > time:nth-child(1)["datetime"]|
|data zakupu produktu|purchase_date|span.user-post__published > time:nth-child(2)["datetime"]|
|ile osób uznało opinię za przydatną|likes|button.vote-yes > span|
|ile osób uznało opinię za nieprzydatną|dislikes|button.vote-no > span|


2. import bibliotek

In [2]:
from flask import Flask, request
import os
import json
import requests
from bs4 import BeautifulSoup

## Funkcja do ekstrakcji danych

In [3]:
def extract(ancestor, selector, attribute = None, return_list = False):
    if return_list:
        if attribute:
            return [tag[attribute].strip() for tag in ancestor.select(selector)]
        return [tag.get_text().strip() for tag in ancestor.select(selector)]
    if selector:
        if attribute:
            try:
                return ancestor.select_one(selector)[attribute].strip()
            except TypeError:
                return None
        try:
            return ancestor.select_one(selector).get_text().strip()
        except AttributeError:
            return None
    if attribute:
        return ancestor[attribute].strip()
    return ancestor.get_text().strip()

3. adres strony

In [4]:
product_id = "84514582"
url = f"https://www.ceneo.pl/{product_id}#tab=reviews"

4. Wysylanie zadan

In [5]:
r = requests.get(url)
r.status_code

200

5. Pobranie opinii

In [6]:
page = BeautifulSoup(r.text, "html.parser")
opinions = page.select("div.js_product-review")

In [7]:
all_opinions = []
while(url):
    response = requests.get(url)
    page = BeautifulSoup(response.text, "html.parser")
    opinions = page.select('div.js_product-review')
    for opinion in opinions:
        single_opinion = {
            key: extract(opinion, *value)
                for key, value in selectors.items()
        }
        all_opinions.append(single_opinion)
    try:
        url = "https://www.ceneo.pl" + extract(page, "a.pagination__next", "href")
    except TypeError:
        break

In [8]:
if not os.path.exists("opinions"):
    os.mkdir("opinions")
with open(f"opinions/{product_id}.json", "w", encoding="UTF-8") as jf:
    json.dump(all_opinions, jf, indent=4, ensure_ascii=False)